In [1]:
%%capture

# get_corpus_path
# get_txt_orig_path
# get_txt_clean_path
%run ../path_manager.ipynb


# CorpusCleaner
%run ../DataCleanerModule.ipynb

## Jupyter.notebook.save_checkpoint()

In [2]:
ENCHANT_INSTALLED

True

In [3]:
import logging

# Misspelling correction

In [4]:
CORPUS_ID = 'IMF'

In [5]:
logging.basicConfig(filename=f'../logs/{CORPUS_ID.lower()}-data-cleaning.log', format='%(asctime)s - %(message)s', level=logging.ERROR)
logger = logging.getLogger(f'{CORPUS_ID.lower()}-cleaning')

In [6]:
TXT_ORIG_DIR = get_txt_orig_path(CORPUS_ID)
TXT_CLEAN_DIR = get_txt_clean_path(CORPUS_ID)

In [7]:
metadata_file = pd.read_csv(os.path.join(get_corpus_path(CORPUS_ID), f'{CORPUS_ID.lower()}_metadata.csv'), low_memory=False)
# metadata_file = metadata_file.dropna(axis=0, subset=['url_txt'])

In [8]:
metadata_file.shape

(14147, 29)

In [9]:
metadata_file.head(2)

,corpus,id,path_original,path_clean,filename_original,year,major_doc_type,doc_type,author,collection,...,adm_region,geo_region,country,wb_lending_instrument,wb_product_line,wb_major_theme,wb_theme,wb_sector,wb_subtopic_src,wb_project_id
0,imf,imf_6cba1c84a4fb1d3dbf4dfbbd011f115eb03d1d68,/NLP/CORPUS/IMF/TXT_ORIG/imf_6cba1c84a4fb1d3db...,/NLP/CORPUS/IMF/TXT_CLEAN/imf_6cba1c84a4fb1d3d...,wp18185.ashx,2018,NaN,NaN,"Nigel A Chalk,Michael Keen,Victoria J Perry",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,imf,imf_b5b7de057454edbaad791b1eb067d6a3cc88cbfd,/NLP/CORPUS/IMF/TXT_ORIG/imf_b5b7de057454edbaa...,/NLP/CORPUS/IMF/TXT_CLEAN/imf_b5b7de057454edba...,cr18255.ashx,2018,NaN,NaN,International Monetary Fund. Asia and Pacific ...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
%%time
cleaner=ParallelCorpusCleaner(
    use_spellchecker=True, use_respeller=True, use_lemmatizer=True, use_spacy=True,
    replacements_plurals_to_singular_file='../whitelists/whitelist_replacements_plurals_to_singular.csv',
    acronyms_file='../whitelists/whitelist_acronyms.csv',
)

cleaner.set_input_folder(TXT_ORIG_DIR)
cleaner.set_output_folder(TXT_CLEAN_DIR)

Doclen 18 < 50 = None
CPU times: user 169 ms, sys: 6 ms, total: 175 ms
Wall time: 174 ms


In [11]:
# %%time
# cleaner=CorpusCleaner(
#     use_spellchecker=True, use_respeller=True, use_lemmatizer=True, use_spacy=True,
#     replacements_plurals_to_singular_file='whitelist_replacements_plurals_to_singular.csv',
# #     logger=logger
# )

# cleaner.set_input_folder(TXT_ORIG_DIR)
# cleaner.set_output_folder(TXT_CLEAN_DIR)

# Run with subset dataset

In [12]:
%%time
# # doclist=['436966.txt', '2002839.txt']

doclist = [f"{id}.txt" for id in metadata_file['id'].sample(n=len(metadata_file), random_state=1029)]

print(len(doclist))

print(pd.datetime.now())
output=cleaner.batch_clean_docs(
    doclist,
#     default_docs_per_worker=10,
    batch_size=96,
    save_docs=True,
    skip_existing=False,
    collect_spell_errors=False)
print(pd.datetime.now())

14147
2018-11-06 06:35:40.686042
Docset 96
Failed lemmatization for imf_041398b287b8f9e64316ca841eb313075a21cf14
Failed lemmatization for imf_0343c2c99febf5c2adde40a15e835d7cd77df68e
Docset 192
No input file: imf_b1ea6114df35a493f8d9679447c6c1acd3541cd7.txt
Failed lemmatization for imf_f49a39b5bfb36804c10c64da0f5f3724ead30ce1
Docset 288
Docset 384
Docset 480
Docset 576
Failed lemmatization for imf_795a94a59b8a97ffceb84433d4df6eac48d86dec
Docset 672
Failed lemmatization for imf_a1e25821ffa7ba4234d5b152a865090a97f771eb
Docset 768
Docset 864
Failed lemmatization for imf_4247ec74701e5ffce391e62cd7d0e031c3505fc1
Docset 960
Docset 1056
Docset 1152
Failed lemmatization for imf_f1c084a95d4a9e2d128ba8671b9e19735c3c3554
Docset 1248
Docset 1344
Docset 1440
Failed lemmatization for imf_63ccf571d5e238603cb2a30e0213a9485b3dc07a
Failed lemmatization for imf_abdf6d072007438da976e58795291b86509c60fe
Docset 1536
Docset 1632
Failed lemmatization for imf_bfad6a46f663bab82f098cb6f379392196931fb5
Docset 172

### Convert the output into a dataframe for easier processing

In [13]:
misc_metadata = pd.DataFrame(output)

In [14]:
# old
misc_metadata.shape

(13984, 7)

In [14]:
# new
misc_metadata.shape

(14129, 7)

Log items that were skipped due to some unexpected errors

In [15]:
misc_metadata[misc_metadata.skipped != ''].skipped.to_csv(f'{CORPUS_ID.lower()}_skipped-items-{pd.datetime.now()}.log')

Generate new attributes based on the `lang` column from `misc_metadata`.

In [16]:
misc_metadata['language_detected'] = misc_metadata.lang.map(lambda x: x[0])
misc_metadata['language_score'] = misc_metadata.lang.map(lambda x: x[1])
misc_metadata = misc_metadata.drop('lang', axis=1)

misc_metadata.index.name = 'id'
misc_metadata = misc_metadata.reset_index()

In [17]:
misc_metadata.head(2)

,id,tokens,text,spell_errors,skipped,exception,write_status,language_detected,language_score
0,imf_000169111a90a98ac66bb6d07e4682d6be8b6ac3,8482,NaN,NaN,,None,True,en,0.999996
1,imf_0004fdded18df40f3680c5a2f343f773d8f18d6f,11825,NaN,NaN,,None,True,en,0.999995


In [18]:
valid_processed_ids = misc_metadata[(misc_metadata.skipped == '') & (misc_metadata.write_status == True)].set_index('id').index
len(valid_processed_ids)

13168

In [19]:
complete_metadata_file = pd.concat([metadata_file.drop_duplicates('id').set_index('id'), misc_metadata.set_index('id')[['language_detected', 'language_score', 'tokens', 'write_status']]], axis=1)
complete_metadata_file = complete_metadata_file.loc[valid_processed_ids]
complete_metadata_file.shape

/home/wb536061/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


(13168, 32)

In [20]:
complete_metadata_file.to_csv(os.path.join(get_corpus_path(CORPUS_ID), f'{CORPUS_ID.lower()}_metadata_complete.csv'))
complete_metadata_file.head(2)

,corpus,path_original,path_clean,filename_original,year,major_doc_type,doc_type,author,collection,title,...,wb_product_line,wb_major_theme,wb_theme,wb_sector,wb_subtopic_src,wb_project_id,language_detected,language_score,tokens,write_status
id,,,,,,,,,,,,,,,,,,,,,
imf_000169111a90a98ac66bb6d07e4682d6be8b6ac3,imf,/NLP/CORPUS/IMF/TXT_ORIG/imf_000169111a90a98ac...,/NLP/CORPUS/IMF/TXT_CLEAN/imf_000169111a90a98a...,_cr12278.ashx,2012,NaN,NaN,International Monetary Fund,NaN,Indonesia : Selected Issues,...,NaN,NaN,NaN,NaN,NaN,NaN,en,0.999996,8482.0,True
imf_0004fdded18df40f3680c5a2f343f773d8f18d6f,imf,/NLP/CORPUS/IMF/TXT_ORIG/imf_0004fdded18df40f3...,/NLP/CORPUS/IMF/TXT_CLEAN/imf_0004fdded18df40f...,_cr0089.ashx,2000,NaN,NaN,International Monetary Fund,NaN,United States : Staff Report for the 2000 Arti...,...,NaN,NaN,NaN,NaN,NaN,NaN,en,0.999995,11825.0,True


In [21]:
complete_metadata_file.shape

(13168, 32)

In [ ]:
# Processing output

In [16]:
s = '''%%time
cleaner = CorpusCleaner(
    use_spellchecker=True, use_respeller=True, use_lemmatizer=True, use_spacy=True,
    replacements_plurals_to_singular_file='whitelist_replacements_plurals_to_singular.csv',
)

cleaner.set_input_folder(TXT_ORIG_DIR)
cleaner.set_output_folder(TXT_CLEAN_DIR)'''

In [21]:
%pprint
print(s)

Pretty printing has been turned OFF
%%time
cleaner=CorpusCleaner(
    use_spellchecker=True, use_respeller=True, use_lemmatizer=True, use_spacy=True,
    replacements_plurals_to_singular_excel='replacements_plurals_to_singular_white_list.xlsx'
)
cleaner.set_input_folder(raw_data_dir)
cleaner.set_output_folder('./processed_files/')
